In [ ]:
import requests
import time
import json


class FbCrawler:
    def __init__(self, page_name):
        self.access_token = "&access_token=" + "EAACEdEose0cBAKvu3lU1PGeOPnSsxeZAwBpaYQsbNDineZB4ZCtfaYVaxTPG9dZAWxlEXxVx6g45cpFspoWwSctqQfr98fAEJu5szTb6yoDAosUJWPb5Jz7rDqLx6BdaPeJl8LnFWPSst4SBOsXBJf9eS4FuGyGbVvHRnIlGsBylO2KbHAXBbGXzIk7ECwB4Us2sgQwyRAZDZD"
        self.page_url = ("https://graph.facebook.com/v3.0/%s/posts?" %page_name)
        self.all_posts = list()
        self.next = None
    
    def clearPosts(self):
        self.all_posts = list()
        
    def getPosts(self, since=None, until=None, next_posts=None):
        time_range = "%s%s" %(""if since is None else "&since="+since, ""if until is None else "&until="+until)
        final_url = (self.page_url+ self.access_token + time_range) if next_posts is None else next_posts
#         print(final_url)
        
        posts_json = json.loads(requests.get(final_url).text)
        for data in posts_json["data"]:
            self.all_posts.append(data)
            
        if ((next_posts is not None) or ((since is not None) and (until is not None))):
            try:
                self.next = posts_json["paging"]["next"]
                self.getPosts(next_posts=self.next)
            except:
                self.next = None
                print("end, count: " + str(len(self.all_posts)))
        
            
    def printPosts(self):
        for element in self.all_posts:
            print(element["created_time"])
    
    def writePosts(self):
        with open("post_"+posts_since+"_"+posts_until+".txt", "w") as writer:
            writer.write(str(self.all_posts))

In [ ]:
posts_since = "2017-1-1"
posts_until = "2017-5-1"

crawler = FbCrawler("DoctorKoWJ")
crawler.clearPosts()
crawler.getPosts(posts_since, posts_until)
crawler.printPosts()
crawler.writePosts()